In [1]:
import csv
import os
import re

from collections import defaultdict

In [2]:
data_dict = {}
with open(r"C:\Users\samys\OneDrive\Рабочий стол\AXL_DATA\DATA\promising_mof\promising_mof_energies\adsorption_energy.txt", "r") as f:
    for line in f:
        key, value = line.strip().split()
        data_dict[key.strip()] = float(value)


print(f"Len : {len(data_dict)} ")

Len : 172659 


In [3]:
root_dir_CO2 = r"C:\Users\samys\OneDrive\Рабочий стол\AXL_DATA\DATA\pristine_CO2"

matching_cif_files_CO2 = []

pattern = re.compile(r"_(\d+)_relaxed\.cif$")

for subdir, dirs, files in os.walk(root_dir_CO2):
    for file in files:
        if file.endswith(".cif") and pattern.search(file):
            name_without_ext = os.path.splitext(file)[0]
            for key in data_dict:
                if name_without_ext.startswith(key):
                    energy = data_dict[key]
                    full_file_path = os.path.join(subdir, file)
                    matching_cif_files_CO2.append((full_file_path, energy))
                    break



In [4]:
root_dir_H2O = r"C:\Users\samys\OneDrive\Рабочий стол\AXL_DATA\DATA\pristine_H2O"

matching_cif_files_H2O = []

pattern = re.compile(r"_(\d+)_relaxed\.cif$")

for subdir, dirs, files in os.walk(root_dir_H2O):
    for file in files:
        if file.endswith(".cif") and pattern.search(file):
            name_without_ext = os.path.splitext(file)[0]
            for key in data_dict:
                if name_without_ext.startswith(key):
                    energy = data_dict[key]
                    full_file_path = os.path.join(subdir, file)
                    matching_cif_files_H2O.append((full_file_path, energy))
                    break



In [5]:
# Сохраняем в CSV
output_full = "matched_cifs_with_energy.csv"
with open(output_full, mode='w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['filename', 'energy'])
    writer.writerows(matching_cif_files_CO2 + matching_cif_files_H2O)



In [6]:
input_csv = "matched_cifs_with_energy.csv"
output_csv = "averaged_energies.csv"

energy_groups = defaultdict(list)

# Читаем CSV
with open(input_csv, newline='', encoding='utf-8') as csvfile:
    reader = csv.reader(csvfile)
    header = next(reader)

    for row in reader:
        filename, energy = row[0], float(row[1])
        base = re.sub(r'(_random)?_\d+_relaxed\.cif$', '', filename)

        energy_groups[base].append(energy)

averaged = [(name, sum(vals) / len(vals)) for name, vals in energy_groups.items()]

with open(output_csv, mode='w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['MOF_name', 'average_energy'])
    writer.writerows(averaged)

print(f"Ready: saved {len(averaged)} raws in '{output_csv}'")

Ready: saved 9183 raws in 'averaged_energies.csv'


In [7]:
print(f'MOFs not in adsorption_energy : {9229 - len(averaged)}')

MOFs not in adsorption_energy : 46
